# Overview #
This script is used to get historical stock prices and other technical data for a list of companies in **ticker** list.

In [ ]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime as dt
import bs4 as bs
import requests
import urllib2

ek.set_app_id('DeNovoQuantFund')

### Stock List ###

In [ ]:
ticker_list = pd.read_csv('/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NASDAQ_mid_2018_review/Fundamental_data_for_top_NASDAQ.csv')

In [ ]:
ticker_list = ticker_list['Instrument'].unique()

In [ ]:
# Get tickers from NASDAQ text file.
NASDAQ_tickers = open("/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/nasdaqlisted.txt","r")
lines = NASDAQ_tickers.readlines()
ticker_list = []
N = len(lines)
for i, line in enumerate(lines):
    if i > 0 and i < N:
        ticker_list.append(line.split("|")[0])
NASDAQ_tickers.close()

## Price and Technical Daily Data 

In [ ]:
technical_fields = [
ek.TR_Field('TR.EVToSales'),
ek.TR_Field('TR.PE'),
ek.TR_Field('TR.PriceToSalesPerShare'),
ek.TR_Field('TR.PriceToBVPerShare'),
ek.TR_Field('TR.PriceToCFPerShare'),
ek.TR_Field('TR.TotalDebtToEBITDA'),
ek.TR_Field('TR.TotalDebtToEV'),
ek.TR_Field('TR.Volume'),
ek.TR_Field('TR.PriceClose'),
ek.TR_Field('TR.PriceClose.date')
]

technical_fields.sort()

## Getting Eikon Tickers
Tickers read from files don't have the proper Eikon extensions. This function tries to find the appropriate Eikon extension for each ticker.

In [ ]:
# Getting correct ticker name
def get_eikon_ticker_name(ticker,f = [ek.TR_Field('TR.RevenuePerShare')], end_date = '2019-01-01'):

    ticker = str(ticker)
    ticker1 = str(ticker)
    
    ticker = ticker1 + '.O'
    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':'2017-01-01', 'EDate': end_date,
                                                                      'Frq': 'D'})
    if err is None:
        return ticker

    ticker = ticker1 + '.OQ'
    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':'2017-01-01', 'EDate': end_date, 
                                                                      'Frq': 'D'})
    if err is None:
        return ticker
    
    ticker = ticker1 + '.TO'
    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':'2017-01-01', 'EDate': end_date, 
                                                                      'Frq': 'D'})
    if err is None:
        return ticker

    ticker = ticker1 + '.N'
    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':'2017-01-01', 'EDate': end_date, 
                                                                      'Frq': 'D'})
    if err is None:
        return ticker
    
    ticker = ticker1
    df_fundamental, err = ek.get_data([ticker], fields= f,parameters={'SDate':'2017-01-01', 'EDate': '2018-01-01'})

    if err is None:
        return ticker

    print('Ticker {} not found'.format(ticker1))
    return None

## Pulling Technical and Daily Data

In [ ]:
# Getting Data for Analysis
def get_technical_and_price_data(ticker, f, start_date1, end_date1):
    
    try:
        # Grab daily technical and pricing data.
        df_tech,err = ek.get_data([ticker], fields= technical_fields,parameters={
            'SDate': start_date1, 'EDate': end_date1, 'Frq': 'D' })
        
        return(df_tech)
    
    except:
        
        print('Error for {}'.format(ticker))
        return(0)

### Getting all stock prices ###

In [ ]:
# Getting Start and End Dates

start_date = dt.datetime(1990,1,1)
end_date = dt.datetime.today()
start_date1 = start_date.strftime('%Y-%m-%d')
end_date1 = end_date.strftime('%Y-%m-%d')

## Creating technical and price dataset

In [ ]:
def write_tech_data(final_file, file_name):
    df = final_file
    df.sort_values(by= ['Instrument','Date'],inplace=True)
    df.dropna(how="all", inplace=True)
    # Write data to csv.
    df.to_csv(file_name)
    print("File Printed")

In [ ]:
final_file = None
cur_file = None
write_file = False
for i,tick in enumerate(ticker_list):
    
    # Write data to csv every 200 iterations.
    # Flag used in case on the 200th iteration the API call fails.
    # Want to make it so the current dataframe is written on the next successsful API call.
    if i > 0 and i % 200 == 0:
        write_file = True
    try:
        tick = get_eikon_ticker_name(tick, end_date = end_date1)
        if tick:
            cur_file = get_technical_and_price_data(tick, technical_fields,
                                                    start_date1, end_date1)

        if (final_file is None and isinstance(cur_file, pd.DataFrame)):
            final_file = cur_file
        
        # Write to csv evey 200 tickers, if the final_file dataframe is populated.
        elif (write_file and isinstance(final_file, pd.DataFrame)):
            write_file = False
            file_name = '/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NASDAQ_2019_Fund_Data/NASDAQ_tech_and_price_data_{}.csv'.format(i)
            write_tech_data(final_file, file_name)
            if isinstance(cur_file, pd.DataFrame):
                final_file = cur_file
            else:
                final_file = None

        elif (isinstance(cur_file, pd.DataFrame)):
            final_file = final_file.append(cur_file, ignore_index = True)

    except urllib2.HTTPError as err:
        print('HTTPError for tick {}'.format(tick))
        print('Error message: {}'.format(err))
    except:
        print('Some error occurred for tick {}.'.format(tick))
    
   
    print(i)

if isinstance(final_file, pd.DataFrame): 
    file_name = '/Users/paindox/Documents/Udemy Apps/DeNovo Quant Fund Stuff/Eikon_Datasets/NASDAQ_2019_Fund_Data/NASDAQ_tech_and_price_data_{}.csv'.format(len(ticker_list))
    write_tech_data(final_file, file_name)
print('DONE!')